 # Install Dependencies

 https://huggingface.co/tiiuae/falcon-40b

In [1]:
!nvidia-smi

Fri Jul 21 11:24:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade

In [4]:
!pip install langchain einops accelerate transformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00


# Import Dependencies

In [5]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import os
import torch

In [6]:
# Check if cuda is available
torch.cuda.is_available()

True

# Build the Pipeline

In [7]:
# Define Model ID
# model_id = "tiiuae/falcon-40b-instruct"

model_id = "tiiuae/falcon-7b-instruct"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Load Model
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir='./workspace/',
    torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", offload_folder="offload")

# Set PT model to inference mode
model.eval()

# Build HF Transformers pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=2000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [8]:
# Test out the pipeline
results = pipeline('who is Atatürk?')
print(results)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': 'who is Atatürk?\nAtatürk was a Turkish revolutionary leader who fought for and successfully achieved independence for the Republic of Turkey from the Byzantine Empire. He later became the President of the Republic.'}]


In [9]:
print(results[0]["generated_text"])

who is Atatürk?
Atatürk was a Turkish revolutionary leader who fought for and successfully achieved independence for the Republic of Turkey from the Byzantine Empire. He later became the President of the Republic.


# Pass it to Langchain with Memory Module

In [10]:
# Setup prompt template
template = PromptTemplate(input_variables=['input'], template='{input}')

# Pass hugging face pipeline to langchain class
llm = HuggingFacePipeline(pipeline=pipeline)

In [11]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, llm=llm)

# conversation.predict(input="Hi there!")

In [12]:

results = conversation.predict(input="Where is Istanbul")

print(results)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 Istanbul is the capital and largest city of Turkey, located on the shores of the Bosporus Straits between the Aegean and Black Seas. It is also a transcontinental city, spanning the continents of Europe and Asia.

Current conversation:
[]
AI: Istanbul is located in Turkey, a transcontinental country spanning both the continents of Europe and Asia.


In [13]:
print(results)

 Istanbul is the capital and largest city of Turkey, located on the shores of the Bosporus Straits between the Aegean and Black Seas. It is also a transcontinental city, spanning the continents of Europe and Asia.

Current conversation:
[]
AI: Istanbul is located in Turkey, a transcontinental country spanning both the continents of Europe and Asia.


In [14]:

results = conversation.predict(input="Can I swim there?")

print(results)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 I do not know.

As an AI language model, I am unable to provide information on the current conditions of natural bodies of water. It is important to consult official and reliable sources for up-to-date safety information.


In [15]:
results

' I do not know.\n\nAs an AI language model, I am unable to provide information on the current conditions of natural bodies of water. It is important to consult official and reliable sources for up-to-date safety information.'

In [ ]:
results = conversation.predict(input="is there hot in summer?")

print(results)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


# Build Gradio App

In [ ]:
# Install Gradio for the UI component
!pip install gradio

In [ ]:
# Import gradio for UI
import gradio as gr

In [ ]:
# Create generate function - this will be called when a user runs the gradio app
def generate(prompt):
    # The prompt will get passed to the LLM Chain!
    return conversation.predict(input=prompt)


In [ ]:
# Define a string variable to hold the title of the app
title = '🦜🔗 Falcon-7b-Instruct'
# Define another string variable to hold the description of the app
description = 'This application demonstrates the use of the open-source `Falcon-7b-Instruct` LLM with memory.'


In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = conversation.predict(input=message)
        chat_history.append((message, bot_message))
        print(chat_history)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)